***Fundamentals of Artificial Intelligence***

> **Lab 4:** *Learning* <br>

> **Performed by:** *Corneliu Catlabuga*, group *FAF-213* <br>

> **Verified by:** Elena Graur, asist. univ.

In [11]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

DATA_PATH = 'data/data.csv'
df = pd.read_csv(DATA_PATH, low_memory=False)

In [12]:
display(df.head())

,Id,EmployeeName,JobTitle,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits,Year,Notes,Agency,Status
0,1,NATHANIEL FORD,GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY,167411.18,0.0,400184.25,NaN,567595.43,567595.43,2011,NaN,San Francisco,NaN
1,2,GARY JIMENEZ,CAPTAIN III (POLICE DEPARTMENT),155966.02,245131.88,137811.38,NaN,538909.28,538909.28,2011,NaN,San Francisco,NaN
2,3,ALBERT PARDINI,CAPTAIN III (POLICE DEPARTMENT),212739.13,106088.18,16452.6,NaN,335279.91,335279.91,2011,NaN,San Francisco,NaN
3,4,CHRISTOPHER CHONG,WIRE ROPE CABLE MAINTENANCE MECHANIC,77916.0,56120.71,198306.9,NaN,332343.61,332343.61,2011,NaN,San Francisco,NaN
4,5,PATRICK GARDNER,"DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)",134401.6,9737.0,182234.59,NaN,326373.19,326373.19,2011,NaN,San Francisco,NaN


In [ ]:
columns = ['BasePay', 'OvertimePay', 'OtherPay', 'Benefits', 'TotalPay', 'TotalPayBenefits']
df[columns] = df[columns].apply(pd.to_numeric, errors='coerce')

display(df[columns].describe().apply(lambda s: s.apply(lambda x: format(x, '.2f'))))

,BasePay,OvertimePay,OtherPay,Benefits,TotalPay,TotalPayBenefits
count,148045.00,148650.00,148650.00,112491.00,148654.00,148654.00
mean,66325.45,5066.06,3648.77,25007.89,74768.32,93692.55
std,42764.64,11454.38,8056.60,15402.22,50517.01,62793.53
min,-166.01,-0.01,-7058.59,-33.89,-618.13,-618.13
25%,33588.20,0.00,0.00,11535.40,36169.00,44065.65
50%,65007.45,0.00,811.27,28628.62,71426.61,92404.09
75%,94691.05,4658.18,4236.07,35566.85,105839.13,132876.45
max,319275.01,245131.88,400184.25,96570.66,567595.43,567595.43


In [ ]:
# Display the number of entries for non-numerical columns
entries = len(df)
num_job_titles = df['JobTitle'].nunique()
num_agencies = df['Agency'].nunique()
num_statuses = df['Status'].nunique()

display(pd.DataFrame({
    'Column': ['Entries', 'Job Titles', 'Agencies', 'Statuses'],
    'Count': [entries, num_job_titles, num_agencies, num_statuses]
}))

,Column,Count
0,Entries,148654
1,Job Titles,2159
2,Agencies,1
3,Statuses,2


In [19]:
# Get unique values for the 'Agency' column and 'Status' column
agencies = df['Agency'].unique()
statuses = df['Status'].unique()

print('Agencies:', agencies)
print('Statuses:', statuses)

Agencies: ['San Francisco']
Statuses: [nan 'PT' 'FT']


In [ ]:

def clean_data(data: pd.DataFrame) -> pd.DataFrame:
    for i in range(len(data)):
        data.loc[i, "JobTitle"] = data.loc[i, "JobTitle"].upper()

    data = data[data["EmployeeName"] != "Not provided"]
    data = data[data["BasePay"] >= 0]

    return data


DATA_PATH = "data/cleaned_data.csv"
try:
    data = read_csv(DATA_PATH, low_memory=False)
except FileNotFoundError:
    print(f"File not found: {DATA_PATH} - Generating data...")
    DATA_PATH = "data/data.csv"
    data = read_csv(DATA_PATH, low_memory=False)
    data = clean_data(data)
    data.to_csv("data/cleaned_data.csv", index=False)